In [21]:
import pandas as pd

In [57]:
data =pd.read_csv('Flight_Booking.csv')

In [33]:
data.head()

,Unnamed: 0,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
1,1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
2,2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956
3,3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955
4,4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955


In [58]:
data.drop(columns='Unnamed: 0',inplace=True)

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300153 entries, 0 to 300152
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   airline           300153 non-null  object 
 1   flight            300153 non-null  object 
 2   source_city       300153 non-null  object 
 3   departure_time    300153 non-null  object 
 4   stops             300153 non-null  object 
 5   arrival_time      300153 non-null  object 
 6   destination_city  300153 non-null  object 
 7   class             300153 non-null  object 
 8   duration          300153 non-null  float64
 9   days_left         300153 non-null  int64  
 10  price             300153 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 25.2+ MB


check null values

In [26]:
data.isnull().sum()

airline             0
flight              0
source_city         0
departure_time      0
stops               0
arrival_time        0
destination_city    0
class               0
duration            0
days_left           0
price               0
dtype: int64

In [27]:
data.shape

(300153, 11)

In [28]:
data.describe()

,duration,days_left,price
count,300153.000000,300153.000000,300153.000000
mean,12.221021,26.004751,20889.660523
std,7.191997,13.561004,22697.767366
min,0.830000,1.000000,1105.000000
25%,6.830000,15.000000,4783.000000
50%,11.250000,26.000000,7425.000000
75%,16.170000,38.000000,42521.000000
max,49.830000,49.000000,123071.000000


In [29]:
data['duration'].dtype

dtype('float64')

In [59]:
#Label encoding

from sklearn.preprocessing import LabelEncoder
le =LabelEncoder()

for i in data.columns:
  if data[i].dtype=='O':
    data[i] =le.fit_transform(data[i])



10

In [63]:
#Feature selection using VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif['index'] = data.columns
vif['vif'] = [variance_inflation_factor(data.values,i) for i in range(len(data.columns))]

In [64]:
vif

,index,vif
0,airline,4.262536
1,source_city,3.225461
2,departure_time,2.893089
3,stops,1.430725
4,arrival_time,4.051207
5,destination_city,3.242476
6,class,9.160322
7,duration,4.967749
8,days_left,4.104448
9,price,6.728865


In [50]:
data.drop(columns='class',inplace=True)

In [62]:
data.drop(columns='flight',inplace=True)

In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300153 entries, 0 to 300152
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   airline           300153 non-null  int64  
 1   source_city       300153 non-null  int64  
 2   departure_time    300153 non-null  int64  
 3   stops             300153 non-null  int64  
 4   arrival_time      300153 non-null  int64  
 5   destination_city  300153 non-null  int64  
 6   class             300153 non-null  int64  
 7   duration          300153 non-null  float64
 8   days_left         300153 non-null  int64  
 9   price             300153 non-null  int64  
dtypes: float64(1), int64(9)
memory usage: 22.9 MB


In [65]:
X =pd.DataFrame(data.iloc[:,:-1])
Y =data['price']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.3)

(300153, 9)

In [66]:
#model building
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(x_train,y_train)

LinearRegression()

In [67]:
pred = lr.predict(x_test)

In [69]:
from sklearn.metrics import *
import numpy as np
np.sqrt(mean_squared_error(y_test,pred))
r2_score(y_test,pred)

0.9054460428680302

In [70]:
#random forest regressor

from sklearn.ensemble import RandomForestRegressor
rgr =RandomForestRegressor()
rgr.fit(x_train,y_train)


RandomForestRegressor()

In [72]:
#rpred =rgr.predict(x_test)
print(np.sqrt(mean_squared_error(y_test,rpred)))

r2_score(y_test,rpred)

2753.7261520434795


0.985172037222673

In [75]:
from sklearn.ensemble import AdaBoostRegressor

In [77]:
ad =AdaBoostRegressor()
ad.fit(x_train,y_train)

AdaBoostRegressor()

In [78]:
apred = ad.predict(x_test)
print(np.sqrt(mean_squared_error(y_test,apred)))

r2_score(y_test,apred)

6018.672777438277


0.9291660511401582

In [79]:
#RF GIVING BEST MODEL
import pickle
pickle.dump(rgr,open('model.pkl','wb'))

In [81]:
pickled_model = pickle.load(open('model.pkl','rb'))